In [1]:
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime

In [2]:
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        # print("True")
        return True
    except:
        conn.close()
        # print("False")
        return False

In [3]:
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [4]:
json_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"json"+os.sep
csv_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"csv"+os.sep

In [5]:
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)

In [6]:
country_name = "india"

#### Let's now make the funciton to shrink the ticker list.

In [7]:
ticker_file_path = "Assets"+os.sep+"Yahoo Ticker Symbols - September 2017.xlsx"
temp_df = pd.read_excel(ticker_file_path)
print("Total stocks:",len(temp_df))
temp_df.head(10)

Total stocks: 106331


,Yahoo Stock Tickers,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,http://investexcel.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticker,Name,Exchange,Category Name,Country,NaN,NaN,NaN
3,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA,NaN,NaN,Samir Khan
4,AAPL,Apple Inc.,NMS,Electronic Equipment,USA,NaN,NaN,simulationconsultant@gmail.com
5,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA,NaN,NaN,NaN
6,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA,NaN,NaN,This ticker symbol list was downloaded from
7,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA,NaN,NaN,http://investexcel.net/all-yahoo-finance-stock...
8,GOOG,Alphabet Inc.,NMS,Internet Information Providers,USA,NaN,NaN,and was updated on 2nd September 2017
9,MO,"Altria Group, Inc.",NYQ,Cigarettes,USA,NaN,NaN,NaN


In [8]:
temp_df = temp_df.drop(temp_df.columns[[5, 6, 7]], axis=1)
headers = temp_df.iloc[2]
df  = pd.DataFrame(temp_df.values[3:], columns=headers)
print("Total stocks:",len(df))
df.head(10)

Total stocks: 106328


2,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA
5,GOOG,Alphabet Inc.,NMS,Internet Information Providers,USA
6,MO,"Altria Group, Inc.",NYQ,Cigarettes,USA
7,DAL,"Delta Air Lines, Inc.",NYQ,Major Airlines,USA
8,AA,Alcoa Corporation,NYQ,Aluminum,USA
9,AXP,American Express Company,NYQ,Credit Services,USA


In [9]:
new_df = df[df["Country"].str.lower().str.contains(country_name.lower()) == True]
print("Total stocks:",len(new_df))
new_df.head(10)

Total stocks: 8984


2,Ticker,Name,Exchange,Category Name,Country
1230,BHARTIARTL.NS,Bharti Airtel Limited,NSI,Wireless Communications,India
1247,ASHOKLEY.NS,Ashok Leyland Limited,NSI,Auto Manufacturers - Major,India
1441,AUROPHARMA.NS,Aurobindo Pharma Limited,NSI,Drugs - Generic,India
1457,AREXMIS.BO,Arex Industries Ltd.,BSE,NaN,India
1586,SANWARIA.NS,Sanwaria Agro Oils Limited,NSI,Farm Products,India
1907,ALMONDZ.NS,Almondz Global Securities Limited,NSI,Investment Brokerage - National,India
1962,ADINATH.BO,Adinath Textiles Ltd,BSE,NaN,India
2897,SBIN.NS,State Bank of India,NSI,Money Center Banks,India
3199,BPCL.NS,Bharat Petroleum Corporation Limited,NSI,Oil & Gas Refining & Marketing,India
3322,MBECL.NS,McNally Bharat Engineering Company Limited,NSI,General Contractors,India


<br/>

#### Saving the list of stcks with tickers with `country_name` in a different `csv` file which can be used later.

In [10]:
new_df.to_csv('Assets'+os.sep+country_name+'.csv', sep=',', index=None)

#### List down the company for which the stocks will be downloaded
<br/>

In [11]:
desired_company_list = ['Tata Motors' ,
                        'Tata Consultancy Services' ]

#### Get the ticker list for the companies user entered in the `desired_company_list`
<br/>

In [12]:
ticker_list=[]
for company in desired_company_list:
    try:
        exact_company_name = (difflib.get_close_matches(company, new_df['Name'])[0])
        ticker_for_the_company = new_df.loc[new_df['Name'] == exact_company_name, 'Ticker'].iloc[0]
        ticker_list.append(ticker_for_the_company)
    except:
        print("Company name "+company+" not found.")

In [13]:
query_urls=[]
for ticker in ticker_list:
    query_urls.append("https://query1.finance.yahoo.com/v8/finance/chart/"+ticker+"?symbol="+ticker+"&period1=0&period2=9999999999&interval=1d&includePrePost=true&events=div%2Csplit")


In [14]:
with Pool(processes=len(query_urls)) as pool:
    pool.starmap(get_historic_price, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("All downloads completed !")

<<<  Historical data of TCS.NS already exists, Updating data...
<<<  Historical data of TATAMOTORS.NS already exists, Updating data...
>>>  Historical data of TCS.NS saved
>>>  Historical data of TATAMOTORS.NS saved
All downloads completed !
